In [0]:
from sklearn.datasets import load_files
from sklearn.feature_extraction.text import CountVectorizer
import numpy as np
import nltk
nltk.download('wordnet')
import re
from nltk.corpus import stopwords
nltk.download('stopwords')
from nltk.stem.porter import PorterStemmer
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
from sklearn import tree
from sklearn import svm
import mglearn
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import VotingClassifier
from sklearn.linear_model import SGDClassifier
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.naive_bayes import MultinomialNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import LinearSVC

# Final Model submmited on Kaggle
### NBSVM trained on all possible trigram features with minimal processing (removal of non-letter character)

In [0]:
reviews_train = load_files("train")
text_train, y_train = reviews_train.data, reviews_train.target
text_train = [doc.replace(b"<br />", b" ") for doc in text_train]  #remove html break
text_train = [doc.decode("utf-8") for doc in text_train]   #decode into utf8
#text_train = [processing(doc) for doc in text_train]    #no processing for best model

print("type of text_train: {}".format(type(text_train)))
print("length of text_train: {}".format(len(text_train)))
print("Samples per class (training): {}".format(np.bincount(y_train)))

In [0]:
import glob
import os
import string

import numpy as np

from sklearn.feature_extraction.text import CountVectorizer

token_pattern = r'\w+|[%s]' % string.punctuation   #remove non-letter characters
vectorizer = CountVectorizer(text_train, ngram_range=(1, 3),
                                 token_pattern=token_pattern,
                                 binary=True)
X_train = vectorizer.fit_transform(text_train)

Based on https://github.com/mesnilgr/nbsvm and https://github.com/Joshua-Chin/nbsvm

In [0]:
from scipy.sparse import spmatrix, coo_matrix
from sklearn.base import BaseEstimator
from sklearn.linear_model.base import LinearClassifierMixin, SparseCoefMixin
from sklearn.svm import LinearSVC

class NBSVM(BaseEstimator, LinearClassifierMixin, SparseCoefMixin):

    def __init__(self, alpha=1, C=1, beta=0.25, fit_intercept=False):
        self.alpha = alpha
        self.C = C
        self.beta = beta
        self.fit_intercept = fit_intercept

    def fit(self, X, y):
        self.classes_ = np.unique(y)
        if len(self.classes_) == 2:
            coef_, intercept_ = self._fit_binary(X, y)
            self.coef_ = coef_
            self.intercept_ = intercept_
        else:
            coef_, intercept_ = zip(*[
                self._fit_binary(X, y == class_)
                for class_ in self.classes_
            ])
            self.coef_ = np.concatenate(coef_)
            self.intercept_ = np.array(intercept_).flatten()
        return self

    def _fit_binary(self, X, y):
        p = np.asarray(self.alpha + X[y == 1].sum(axis=0)).flatten()
        q = np.asarray(self.alpha + X[y == 0].sum(axis=0)).flatten()
        r = np.log(p/np.abs(p).sum()) - np.log(q/np.abs(q).sum())
        b = np.log((y == 1).sum()) - np.log((y == 0).sum())

        if isinstance(X, spmatrix):
            indices = np.arange(len(r))
            r_sparse = coo_matrix(
                (r, (indices, indices)),
                shape=(len(r), len(r))
            )
            X_scaled = X * r_sparse
        else:
            X_scaled = X * r

        lsvc = LinearSVC(
            C=self.C,
            fit_intercept=self.fit_intercept,
            max_iter=10000
        ).fit(X_scaled, y)

        mean_mag =  np.abs(lsvc.coef_).mean()

        coef_ = (1 - self.beta) * mean_mag * r + \
                self.beta * (r * lsvc.coef_)

        intercept_ = (1 - self.beta) * mean_mag * b + \
                     self.beta * lsvc.intercept_

        return coef_, intercept_

In [0]:
nbsvm = NBSVM()
param_grid = {'C': [1, 10, 100] }  #best at C = 1
grid = GridSearchCV(nbsvm, param_grid, cv=5, verbose = 30)
grid.fit(X_train, y_train)

In [0]:
print("Best cross-validation score: {:.4f}".format(grid.best_score_))
print("Best parameters:\n{}".format(grid.best_params_))
grid.grid_scores_

In [0]:
best_model = grid.best_estimator_

# Bernoulli Naive Bayes

In [0]:
import numpy as np
import pandas as pd
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score
from sklearn.datasets import load_files

In [0]:
reviews_train = load_files("train")

text_train, y_train = reviews_train.data, reviews_train.target
text_train = [doc.replace(b"<br />", b" ") for doc in text_train]  #remove html break
text_train = [doc.decode("utf-8") for doc in text_train]   #decode into utf8


from sklearn.model_selection import train_test_split
#Here we conduct the training and test set split.
X_train0, X_val, y_train0, y_val = train_test_split(text_train, y_train, test_size=0.2, random_state=42)

In [0]:
def preprocess_string(str_arg):

        preprocessed_str=re.sub('[^a-z\\s]+',' ',str_arg,flags=re.IGNORECASE) #every char except alphabets is replaced
        preprocessed_str=re.sub('(\\s+)',' ',preprocessed_str) 
        preprocessed_str=preprocessed_str.lower() 
    
        return preprocessed_str # returning the preprocessed string 

In [0]:
class NaiveBayes:
    def __init__(self,unique_classes,alpha=1.0):
        self.alpha = alpha
        self.classes=unique_classes 
    def addToBow(self,example,index): #Adding to the bag of words
        
        if isinstance(example,np.ndarray): example=example[0]
     
        for token_word in example.split(): #for every word in preprocessed example
          
            self.bow_dicts[index][token_word]+=1 #increment in its count
            
            
    
    
    def train(self,dataset,labels):    
        self.examples=dataset
        self.labels=labels
        self.bow_dicts=np.array([defaultdict(lambda:0) for index in range(self.classes.shape[0])])
        
        if not isinstance(self.examples,np.ndarray): self.examples=np.array(self.examples)
        if not isinstance(self.labels,np.ndarray): self.labels=np.array(self.labels)
            
        #constructing BoW for each category
        for cat_index,cat in enumerate(self.classes):
          
            all_cat_examples=self.examples[self.labels==cat] 
            
            preprocessed_examples=[preprocess_string(cat_example) for cat_example in all_cat_examples]
            
            preprocessed_examples=pd.DataFrame(data=preprocessed_examples)
            
            #now costruct BoW of this particular category
            np.apply_along_axis(self.addToBow,1,preprocessed_examples,cat_index)
      
        prob_classes=np.empty(self.classes.shape[0])
        all_words=[]
        cat_word_counts=np.empty(self.classes.shape[0])
        for cat_index,cat in enumerate(self.classes):
            prob_classes[cat_index]=np.sum(self.labels==cat)/float(self.labels.shape[0]) 
            #Calculating total counts of all the words of each class 
            #Here we applied the Laplace Smoothing
            count=list(self.bow_dicts[cat_index].values())
            cat_word_counts[cat_index]=np.sum(np.array(list(self.bow_dicts[cat_index].values())))+1 # |v| is remaining to be added                           
            all_words+=self.bow_dicts[cat_index].keys() 
            
        self.vocabulary=np.unique(np.array(all_words))
        self.vocab_length=self.vocabulary.shape[0]
                                  
        #computing denominator value                                      
        denominator=np.array([cat_word_counts[cat_index]+self.vocab_length+1 for cat_index,cat in enumerate(self.classes)])                                                                          
        
        self.cats_info=[(self.bow_dicts[cat_index],prob_classes[cat_index],denominator[cat_index]) for cat_index,cat in enumerate(self.classes)]                               
        self.cats_info=np.array(self.cats_info)    
    
    
    
    def getExampleProb(self,test_example):                                                                              
        likelihood_probability=np.zeros(self.classes.shape[0]) #to store probability w.r.t each class
        for cat_index,cat in enumerate(self.classes):                   
            for test_token in test_example.split():                                                     
                test_token_counts=self.cats_info[cat_index][0].get(test_token,0)+1                            
                test_token_prob=test_token_counts/float(self.cats_info[cat_index][2])                             
                likelihood_probability[cat_index]+=np.log(test_token_prob) #Taking log to prevent underflow 
        post_probability=np.empty(self.classes.shape[0])
        for cat_index,cat in enumerate(self.classes):
            post_probability[cat_index]=likelihood_probability[cat_index]+np.log(self.cats_info[cat_index][1])                                   
        return post_probability
    
    
    
    
    def test(self,test_set):    
        predictions=[] #to store prediction of each test example
        for example in test_set:                                
            processed_set=preprocess_string(example)                                 
            post_probability=self.getExampleProb(processed_set)
            predictions.append(self.classes[np.argmax(post_probability)]) #Here we simply pick the largest element to add to the predictions.              
        return np.array(predictions)
                                              

In [0]:
nb = NaiveBayes(np.unique(y_train0))
print ("Training")
nb.train(X_train0,y_train0) 
print ("Completed")

In [0]:
y_pred = nb.test(X_val) 

In [0]:
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score
accuracy_score(y_val, y_pred)

# Generate Prediction Submission

In [0]:
import pandas as pd

def read_testfile(folder):
  df1 = pd.DataFrame()

    
  for file_num in range(0,25000):

    temp_name = folder+'/'+str(file_num)+'.txt'
    fileHandle = open (temp_name, encoding="utf8" )
    str2 = fileHandle.read()
      
    #str2 = processing(str2)

    #file_name = re.sub("[^0-9]", "", file_name)
    
    df1 = df1.append([[file_num, str2]],ignore_index=True)
      
  df1.columns = ['Id', 'comment'] 
  df1 = df1.sort_values(by=['Id'])

  return df1


df_test = read_testfile('test')
X_test = np.array(list(df_test['comment']))

In [0]:
#best model submitted so far, feel free to try your models in here; bi- or tri- gram does not make huge difference
pipe = make_pipeline(CountVectorizer(text_train, ngram_range=(1, 3),
                                 token_pattern=token_pattern,
                                 binary=True), 
                     NBSVM(C =1))

pipe.fit(text_train, y_train)

test_prediction = pipe.predict(X_test)

In [0]:
submission = pd.DataFrame({'Category': test_prediction, 
                           'Id': df_test['Id']})
cols = submission.columns.tolist()
cols = cols[-1:] + cols[:-1]
submission = submission[cols]
submission = submission.sort_values(by=['Id'])
submission.to_csv('submission.csv', index=False)

submission.head(100)

# Additional Experiments

In [0]:
def processing(input_str):
  stop = set(stopwords.words('english'))
  
  # Remove characters that are not letters
  input_str1 = re.sub("[^a-zA-Z]", " ", input_str) 

  # Lowercase to string and split to words
  input_words = input_str1.lower().split()

  # lemmatization of words
  input_words1 = [WordNetLemmatizer().lemmatize(words) for words in input_words]
  
  # Convert to a string again
  output_str = ( " ".join(input_words1))
  
  return output_str

In [0]:
reviews_train = load_files("train")
text_train, y_train = reviews_train.data, reviews_train.target
text_train = [doc.replace(b"<br />", b" ") for doc in text_train]  #remove html break
text_train = [doc.decode("utf-8") for doc in text_train]   #decode into utf8
text_train = [processing(doc) for doc in text_train]    #perform general processing

print("type of text_train: {}".format(type(text_train)))
print("length of text_train: {}".format(len(text_train)))
print("Samples per class (training): {}".format(np.bincount(y_train)))

## Raw Count

In [0]:
# produce count vector and transform over the data into a feature matrix
vect = CountVectorizer(min_df = 5).fit(text_train)
X_train = vect.transform(text_train)
print("X_train:\n{}".format(repr(X_train)))


#print some feature names to check if you want
feature_names = vect.get_feature_names()
#print("Features 20010 to 20030:\n{}".format(feature_names[20010:20030]))

In [0]:
param_grid = {'C': [0.001, 0.01, 0.1, 1, 10]}
grid = GridSearchCV(LogisticRegression(), param_grid, cv=5)
grid.fit(X_train, y_train)
print("Best cross-validation score: {:.2f}".format(grid.best_score_))
print("Best parameters: ", grid.best_params_)

## TF-IDF

In [0]:
# produce count vector and transform over the data into a feature matrix
vect = TfidfVectorizer(min_df = 5).fit(text_train)
X_train = vect.transform(text_train)
print("X_train:\n{}".format(repr(X_train)))


#print some feature names to check if you want
feature_names = vect.get_feature_names()
#print("Features 20010 to 20030:\n{}".format(feature_names[20010:20030]))

In [0]:
param_grid = {'C': [0.001, 0.01, 0.1, 1, 10]}
grid = GridSearchCV(LogisticRegression(), param_grid, cv=5)
grid.fit(X_train, y_train)
print("Best cross-validation score: {:.2f}".format(grid.best_score_))
print("Best parameters: ", grid.best_params_)

##TF-IDF + ngram

In [0]:
pipe = make_pipeline(TfidfVectorizer(min_df=5), LogisticRegression())
# running the grid search takes a long time because of the
# relatively large grid and the inclusion of trigrams
param_grid = {"logisticregression__C": [0.001, 0.01, 0.1, 1, 10, 100],
"tfidfvectorizer__ngram_range": [(1, 1), (1, 2), (1, 3)]}
grid = GridSearchCV(pipe, param_grid, cv=5)
grid.fit(text_train, y_train)
print("Best cross-validation score: {:.2f}".format(grid.best_score_))
print("Best parameters:\n{}".format(grid.best_params_))

In [0]:
import matplotlib as plt
# extract scores from grid_search
scores = grid.cv_results_['mean_test_score'].reshape(-1, 3).T
# visualize heat map
heatmap = mglearn.tools.heatmap(
scores, xlabel="C", ylabel="ngram_range", cmap="viridis", fmt="%.3f",
xticklabels=param_grid['logisticregression__C'],
yticklabels=param_grid['tfidfvectorizer__ngram_range'])

In [0]:
# extract feature names and coefficients
vect = grid.best_estimator_.named_steps['tfidfvectorizer']
feature_names = np.array(vect.get_feature_names())
    coef = grid.best_estimator_.named_steps['logisticregression'].coef_
mglearn.tools.visualize_coefficients(coef, feature_names, n_top_features=40)

In [0]:
# find 3-gram features
mask = np.array([len(feature.split(" ")) for feature in feature_names]) == 3
# visualize only 3-gram features
mglearn.tools.visualize_coefficients(coef.ravel()[mask],
feature_names[mask], n_top_features=40)

## Linear SVC

In [0]:
pipe = make_pipeline(TfidfVectorizer(min_df=5), 
                     LinearSVC())
param_grid = {"linearsvc__C": [0.1, 1, 10, 100],
"tfidfvectorizer__ngram_range": [(1, 2), (1, 3)]}
grid = GridSearchCV(pipe, param_grid, cv=5)
grid.fit(text_train, y_train)
print("Best cross-validation score: {:.3f}".format(grid.best_score_))
print("Best parameters:\n{}".format(grid.best_params_))

## Multinomial NB

In [0]:
pipe = make_pipeline(TfidfVectorizer(min_df=5), 
                     MultinomialNB())
param_grid = {"multinomialnb__alpha": [0.1, 1, 10, 100],
"tfidfvectorizer__ngram_range": [(1, 2), (1, 3)]}
grid = GridSearchCV(pipe, param_grid, cv=5)
grid.fit(text_train, y_train)
print("Best cross-validation score: {:.3f}".format(grid.best_score_))
print("Best parameters:\n{}".format(grid.best_params_))

## Word Embedding

In [0]:
#Train a new word embedding model
import gensim
from nltk.tokenize import word_tokenize
from gensim.models import Phrases

import nltk
nltk.download('punkt')

#this is adapted to incorporat bigram
text_train_tokenized = [word_tokenize(doc) for doc in text_train]

#train the model with vec size 100
model = gensim.models.Word2Vec(text_train_tokenized, size=100, min_count=1, window=10, hs=0,negative =5, sample =1e-4,iter=20,workers=-1)
w2v = dict(zip(model.wv.index2word, model.wv.syn0))

In [0]:
#word embedding vectorizer with fit and transform methods
import collections

class TfidfEmbeddingVectorizer(object):
    def __init__(self, word2vec):
        self.word2vec = word2vec
        self.word2weight = None
        self.dim = len(word2vec.values())

    def fit(self, X, y):
        tfidf = TfidfVectorizer(analyzer=lambda x: x)
        tfidf.fit(X)
        max_idf = max(tfidf.idf_)
        self.word2weight = collections.defaultdict(
            lambda: max_idf,
            [(w, tfidf.idf_[i]) for w, i in tfidf.vocabulary_.items()])

        return self

    def transform(self, X):
        return np.array([
                np.mean([self.word2vec[w] * self.word2weight[w]
                         for w in words if w in self.word2vec] or
                        [np.zeros(self.dim)], axis=0)
                for words in X
            ])

In [0]:
#data is prepared, feel free to try those on any model but I don't see improvement yet; maybe you will be the lucky one
vect = TfidfEmbeddingVectorizer(w2v).fit(text_train_tokenized,y_train)
X_train = vect.transform(text_train_tokenized)


## I dont suggest trying trees cuz it takes forever to train;
## this word embedding methods seems to work well with deep learning models as someone suggests


## NBLR

In [0]:
vect = TfidfVectorizer(min_df = 5, ngram_range = (1,3)).fit(text_train)
X_train = vect.transform(text_train)

In [0]:
from sklearn.base import BaseEstimator, ClassifierMixin
from sklearn.utils.validation import check_X_y, check_is_fitted
from sklearn.linear_model import LogisticRegression
from scipy import sparse
class NbLrClassifier(BaseEstimator, ClassifierMixin):
    def __init__(self, C=1.0, dual=False, n_jobs=1):
        self.C = C
        self.dual = dual
        self.n_jobs = n_jobs

    def predict(self, x):
        # Verify that model has been fit
        check_is_fitted(self, ['_r', '_clf'])
        return self._clf.predict(x.multiply(self._r))

    def predict_proba(self, x):
        # Verify that model has been fit
        check_is_fitted(self, ['_r', '_clf'])
        return self._clf.predict_proba(x.multiply(self._r))

    def fit(self, x, y):
        # Check that X and y have correct shape
        x, y = check_X_y(x, y, accept_sparse=True)

        def pr(x, y_i, y):
            p = x[y==y_i].sum(0)
            return (p+1) / ((y==y_i).sum()+1)

        self._r = sparse.csr_matrix(np.log(pr(x,1,y) / pr(x,0,y)))
        x_nb = x.multiply(self._r)
        self._clf = LogisticRegression(C=self.C, dual=self.dual, n_jobs=self.n_jobs).fit(x_nb, y)
        return self

In [0]:
nbsvm = NbLrClassifier()
param_grid = {'C': [0.1,1,10,100] }
grid = GridSearchCV(nbsvm, param_grid, cv=5, verbose = 30)
grid.fit(X_train, y_train)
print("Best cross-validation score: {:.4f}".format(grid.best_score_))
print("Best parameters:\n{}".format(grid.best_params_))
grid.grid_scores_

## Voting Classifier

In [0]:
clf1 = NbLrClassifier(dual=True, n_jobs=-1)
clf2 = MultinomialNB()
clf3 = LogisticRegression()

In [0]:
vlcf = VotingClassifier(estimators=[('mnb', clf2), ('nb', clf1), ('lr', clf3)], voting='soft')
param_grid = {'mnb__alpha': [0.01, 0.1, 1, 10, 100], 'nb__C': [0.01, 0.1, 1, 10, 100], 'lr__C': [0.01, 0.1, 1, 10, 100]}
grid = GridSearchCV(vlcf, param_grid, cv=5, verbose = 30)
grid.fit(X_train, y_train)
print("Best cross-validation score: {:.3f}".format(grid.best_score_))
print("Best parameters:\n{}".format(grid.best_params_))
grid.grid_scores_

In [0]:
clf1 = NbSvmClassifier(C = 100, dual=True, n_jobs=-1)
clf2 = MultinomialNB(alpha = 0.01)
clf3 = LogisticRegression(C = 100,dual=True)
vlcf = VotingClassifier(estimators=[('mnb', clf2),('nb', clf1),('lr',clf3)], voting='soft')
param_grid = {'weights': [(0.3,0.6,0.1),(0.2,0.7,0.1),(0.3,0.5,0.2),(0.2,0.5,0.3),(0.1,0.6,0.3),(0.1,0.7,0.2)]}
grid = GridSearchCV(vlcf, param_grid, cv=5, verbose = 30)
grid.fit(X_train, y_train)